## Exercises I

In [1]:
import pandas as pd
import numpy as np
import env

Use your function to obtain a connection to the employees database.

In [ ]:
pd.read_sql('SELECT * FROM employees LIMIT 5', env.get_db_url('employees'))

a. Intentionally make a typo in the database url. What kind of error message do you see?

In [ ]:
# There is an access denied message becasue of the mispelled db name
pd.read_sql('SELECT * FROM employees LIMIT 5', env.get_db_url('employes'))

b. Intentionally make an error in your SQL query. What does the error message look like?

In [ ]:
# it says I have an error in my sql syntax
pd.read_sql('SELECT * FROM employees LMIT 5', env.get_db_url('employees'))

Read the employees and titles tables into two separate DataFrames.

In [ ]:
emp_df = pd.read_sql('SELECT * FROM employees', env.get_db_url('employees'))

In [ ]:
titles_df = pd.read_sql('SELECT * FROM titles', env.get_db_url('employees'))

How many rows and columns do you have in each DataFrame? Is that what you expected?

In [ ]:
emp_df.shape

In [ ]:
titles_df.shape

Display the summary statistics for each DataFrame.

In [ ]:
emp_df.describe(), emp_df.info()

In [ ]:
titles_df.info(), titles_df.describe()

How many unique titles are in the titles DataFrame?

In [ ]:
len(titles_df.title.unique())

In [ ]:
len(titles_df.title.value_counts())

What is the oldest date in the to_date column?

In [ ]:
titles_df.to_date.min()

What is the most recent date in the to_date column?

In [ ]:
# including currrently employed
titles_df.to_date.max()

In [ ]:
# excluding currently employed people
titles_df.to_date[titles_df.to_date != titles_df.to_date.max()].max()

## Exercises II

Copy the users and roles DataFrames from the examples above.

In [ ]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


What is the result of using a right join on the DataFrames?

In [ ]:
#roles = roles.rename(columns={'id':'role_id', 'name':'role'})
pd.merge(users, roles, how='right', on='role_id')

What is the result of using an outer join on the DataFrames?

In [ ]:
pd.merge(users, roles, how='outer', on='role_id')

What happens if you drop the foreign keys from the DataFrames and try to merge them?

In [ ]:
# if you drop the foreign keys there is no way to correctly merge to roles with the users. If you try to merge with the users 'id' there is no relationship between the users 'id' and the roles 'id'

Load the mpg dataset from PyDataset.

In [ ]:
from pydataset import data
mpg = data('mpg')

Output and read the documentation for the mpg dataset.

In [ ]:
data('mpg', show_doc=True)

How many rows and columns are in the dataset?

In [ ]:
mpg.shape

Check out your column names and perform any cleanup you may want on them.

In [ ]:
## change drv to drive
## change displ to displacement
# mpg = mpg.rename(columns={'drv':'drive', 'displ':'displacement', 'cyl':'cylinders'})

Display the summary statistics for the dataset.

In [ ]:
mpg.describe()

How many different manufacturers are there?

In [ ]:
mpg.manufacturer.nunique()

How many different models are there?

In [ ]:
mpg.model.nunique()

Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

In [ ]:
#mpg['mileage_diff'] = mpg.hwy - mpg.cty
mpg

Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.

In [ ]:
# mpg['avg_mileage'] = mpg[['cty', 'hwy']].agg('mean', axis=1)
mpg

Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.

In [ ]:
mpg['is_automatic'] = mpg.trans.apply(lambda x:'auto' in x)
# mpg[['trans', 'is_automatic']]
# mpg.trans.str.contains('auto')
mpg.trans.str.startswith('auto')

Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?

In [ ]:
mpg['avg_mileage'].groupby(mpg.manufacturer).agg('mean').nlargest(1)
mpg.groupby(mpg.manufacturer).avg_mileage.mean()
mpg.avg_mileage.groupby(mpg.manufacturer).mean().nlargest(1)

Do automatic or manual cars have better miles per gallon?

In [ ]:
mpg['avg_mileage'].groupby(mpg.is_automatic).agg('mean')
mpg.groupby('is_automatic').avg_mileage.mean()

## Exercises III

Use your get_db_url function to help you explore the data from the chipotle database.

In [29]:
df = pd.read_sql('SELECT * FROM orders', env.get_db_url('chipotle'))

What is the total price for each order?

In [23]:
# df.item_price = df.item_price.apply(lambda x: float(x.replace('$', '')))
# df.quantity = df.quantity.apply(float)

df['item_order_price'] = df.quantity * df.item_price

df[['order_id', 'item_order_price']].groupby('order_id').sum()

,item_order_price
order_id,
1,11.56
2,33.96
3,12.67
4,21.00
5,13.70
...,...
1830,23.00
1831,12.90
1832,13.20


What are the most popular 3 items?

In [24]:
df.item_name.value_counts().head(3)

Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: item_name, dtype: int64

Which item has produced the most revenue?

In [30]:
# item count * the item price

df.item_name.value_counts() * df.item_pric
# df.groupby('item_name').item_order_price.agg('sum').sort_values(ascending=False).nlargest(1)



0                      NaN
1                      NaN
2                      NaN
3                      NaN
4                      NaN
                      ... 
Veggie Burrito         NaN
Veggie Crispy Tacos    NaN
Veggie Salad           NaN
Veggie Salad Bowl      NaN
Veggie Soft Tacos      NaN
Length: 4672, dtype: object

Join the employees and titles DataFrames together.

In [27]:
df = pd.read_sql('SELECT * FROM employees JOIN titles USING (emp_no)', env.get_db_url('employees'))

For each title, find the hire date of the employee that was hired most recently with that title.

In [15]:
df.pivot_table(index='title', values='hire_date', aggfunc='max')

,hire_date
title,
Assistant Engineer,1999-12-24
Engineer,2000-01-28
Manager,1992-02-05
Senior Engineer,2000-01-01
Senior Staff,2000-01-13
Staff,2000-01-12
Technique Leader,1999-12-31


Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

In [16]:
df = pd.read_sql('''SELECT *
FROM titles
	JOIN dept_emp USING (emp_no)
	JOIN departments USING (dept_no);''', env.get_db_url('employees'))





In [18]:
pd.crosstab(df.dept_name, df.title)

title,Assistant Engineer,Engineer,Manager,Senior Engineer,Senior Staff,Staff,Technique Leader
dept_name,,,,,,,
Customer Service,298,2362,4,2027,13925,16150,309
Development,7769,58135,2,49326,1247,1424,7683
Finance,0,0,2,0,12139,13929,0
Human Resources,0,0,2,0,12274,14342,0
Marketing,0,0,2,0,13940,16196,0
Production,6445,49649,4,42205,1270,1478,6557
Quality Management,1831,13852,4,11864,0,0,1795
Research,378,2986,2,2570,11637,13495,393
Sales,0,0,2,0,36191,41808,0
